## <font color='darkblue'>Homework1</font>
Below is the requirement of [**Homework1**](https://github.com/johnklee/oo_dp_lesson/issues/1):
* **針對 VIP 結帳的 policy:**
    * If the total price is less than 100, keep the original price: input=99, output=99
    * If the total price is between \[100~200`], take 10% off: input=200, output=180
    * If the total price is between \[201~500`], take 20% off: input=500, output=400
    * If the total price is greater than 500, take 40% off. If the calculated price is less than 400, use 400:
      - input=600, output=max(400, 600*0.6)=max(400, 360)=400
      - input=1000, output=600
* **針對 MEMBER 結帳的 policy:**
    * If the customer is a member: Always take 10% off.
* **針對 NORMAL 結帳的 policy:** 
    * no discount at all.

In [12]:
from enum import Enum
from typing import List

#######################
# Task2
#######################
class CustomerType(Enum):
        NORMAL=0
        MEMBER=1
        VIP=3

### <font color='darkgreen'>實作類別 CounterV2</font>
底下是實作類別 **CounterV2** 的 skeleton:
```python
class CounterV2:
  def __init__(self):
    pass

  def count(self, prices:List[int], customer_type:CustomerType):
    '''Calculate the final price based on given list of price according to customer type

    - If the customer is VIP: Use same discount policy as Counter
    - If the customer is a member: Always take 10% off.
    - Otherwise, no discount at all.
    '''
    # TBD
    pass
```

### <font color='darkgreen'>實作思路</font>
不過是 VIP, MEMBER 還是 NORMAL 的客戶, 計算的過程雖然不同 (Policy), 但是 輸入(Input) 與 輸出(Output) 的結構與類型卻是一致的:
![14.PNG](images/14.PNG)
<br/>

這個結構非常符合 design pattern 中的 [**Strategy pattern**](https://en.wikipedia.org/wiki/Strategy_pattern), 來看一下它的 class diagram:
![class diagram](images/W3sDesign_Strategy_Design_Pattern_UML.jpg)
<br/>

上面的類別可以理解成:
* **Context**: 等同類別 **CounterV2**. 利用 Policy 計算折扣.
* **Strategy**: 等同類別 **Policy**. 用來計算折扣.

根據這個概念, 接著實作如下:

In [2]:
import math
from abc import abstractmethod, ABC
from typing import List


class Policy(ABC):
    @abstractmethod
    def count(self, prices:List[int]):
        raise NotImplemented()
        
        
class VipPolicy(Policy):
    def count(self, prices:List[int]):
        price_sum = sum(prices)
        if price_sum < 100:
            return price_sum
        elif price_sum <= 200:
            return math.floor(0.9 * price_sum)
        elif price_sum <= 500:
            return math.floor(0.8 * price_sum)
        else:
            return max(400, math.floor(0.6 * price_sum))
        
class MemberPolicy(Policy):
    def count(self, prices:List[int]):
        price_sum = sum(prices)
        return math.floor(price_sum * 0.9)


class NormalPolicy(Policy):
    def count(self, prices:List[int]):    
        return sum(prices)

因為我們將折扣透過 Policy 封裝起來, 原先的類別 **CounterV2** 的實作變得簡單: 

In [3]:
class CounterV2:
    def __init__(self):
        pass

    def count(self, prices:List[int], customer_type:CustomerType):
        '''Calculate the final price based on given list of price according to customer type
        - If the customer is VIP: Use same discount policy as Counter
        - If the customer is a member: Always take 10% off.
        - Otherwise, no discount at all.
        '''
        if customer_type == CustomerType.NORMAL:
            return NormalCount().count(prices)
        elif customer_type == CustomerType.MEMBER:
            return MemberPolicy().count(prices)
        else:
            return VipPolicy().count(prices) 

上面的 if/else 事實上可以再優化, 之後在介紹其他 design pattern 時會再說明:

In [4]:
class CounterV2:
    def __init__(self):
        pass

    def count(self, prices:List[int], customer_type:CustomerType):
        '''Calculate the final price based on given list of price according to customer type
        - If the customer is VIP: Use same discount policy as Counter
        - If the customer is a member: Always take 10% off.
        - Otherwise, no discount at all.
        '''
        return globals()[customer_type.name.capitalize()+"Policy"]().count(prices)        

接著我們便可使用如下:

In [5]:
cnt = CounterV2()
prices = [600]
print(f"prices={prices}:")
for ct in CustomerType:
    print(f"\t{ct.name} should pay {cnt.count(prices, ct)}")

prices=[600]:
	NORMAL should pay 600
	MEMBER should pay 540
	VIP should pay 400


## <font color='darkblue'>Command Pattern</font>
在物件導向程式設計的範疇中，[**命令模式**](https://zh.wikipedia.org/wiki/%E5%91%BD%E4%BB%A4%E6%A8%A1%E5%BC%8F)（英語：Command pattern）是一種設計模式，它嘗試以物件來代表實際行動。**命令物件 可以把 行動** (action) **及其參數封裝起來，於是這些行動可以被**：
* 重複多次
* 取消（如果該物件有實作的話）
* 取消後又再重做

這些都是現代大型應用程式所必須的功能，即「復原」及「重複」。除此之外，可以用命令模式來實作的功能例子還有：
* 交易行為
* 進度列
* 精靈
* 使用者介面按鈕及功能表項目
* 執行緒 pool
* 巨集收錄

### <font color='darkgreen'>Class Diagram</font>
* <font color='blue'><b>Invoker</b></font>: 寄存建立的 <b>Command</b> 物件清單, 並將來自 <b>Client</b> 的呼叫轉成執行對應的 <b>Command</b> 上的函數.
* <font color='blue'><b>Command</b></font>: 執行動作的介面.
* <font color='blue'><b>ConcreteCommand</b></font>: 執行動作的實作
* <font color='blue'><b>Client</b></font>: 決定 <b>Invoker</b> 需要建立的 <b>Command</b> 物件清單
* <font color='blue'><b>Receiver</b></font>: 執行 <b>Command</b> 上的函數時會被呼叫的物件總稱.

![command class diagram](images/16.PNG)
<br/>
上面是學術或上課老師的講法, 接下來是類比成生活事件 ([source](https://ithelp.ithome.com.tw/articles/10246587)):
> 老闆 (Client) 將任務封裝成備忘錄 (Command)，然後交付給秘書(Invoker), 接著 秘書(Invoker) 便根據 備忘錄的工作事項 (Command) 分派任務給員工(Receiver)。如此一來老闆不需要知道是哪個員工執行，只需要秘書回報任務結果即可。

### <font color='darkgreen'>使用範例</font>
考慮 我們 (<font color='blue'><b>User</b></font>) 要設計一個萬能遙控器 (<font color='blue'><b>RemoteController</b></font>), 它目前能夠:
* 關/開 燈 (<font color='blue'><b>Lamp</b></font>)

如果套到 [<b>Command pattern</b>](https://zh.wikipedia.org/wiki/%E5%91%BD%E4%BB%A4%E6%A8%A1%E5%BC%8F):

In [6]:
from abc import abstractmethod, ABC


class Lamp(object):
    def __init__(self, state=0):
        self.state = 0  # 0:Off; 1:On
        
    def on(self):
        if self.state == 0:
            self.state = 1
            print("Turn on light")            
        else:
            print("Light is already on!")
            
    def off(self):
        if self.state == 0:
            print("Light is already off!")
        else:
            self.state = 0
            print("Turn off light")
            

class Command(ABC):
    def do(self):
        raise NotImplemented()
        

class CommandTurnOnLight(Command):
    def __init__(self, lamp):
        self.lamp = lamp
        
    def do(self):
        self.lamp.on()
        
        
class CommandTurnOffLight(Command):
    def __init__(self, lamp):
        self.lamp = lamp
        
    def do(self):
        self.lamp.off()
        
        
class RemoteController:
    def __init__(self):
        self.cmd_dict = {}
        
    def regr_command(self, name:str, cmd:Command):
        self.cmd_dict[name] = cmd
        
    def __getattr__(self, name):
        return self.cmd_dict[name].do()
    
    
class User:
    def __init__(self):
        self.rctr = RemoteController()
        lamp = Lamp()
        self.rctr.regr_command(
            'turn_on_light',
            CommandTurnOnLight(lamp)
        )
        self.rctr.regr_command(
            'turn_off_light',
            CommandTurnOffLight(lamp)
        )
    
    def work(self):
        self.rctr.turn_on_light
        self.rctr.turn_off_light

接著來測試一下 <font color='blue'><b>User</b></font>

In [7]:
u = User()
u.work()

Turn on light
Turn off light


接著如果我們希望這個 萬能遙控器 (<font color='blue'><b>RemoteController</b></font>) 也可以開關 電視 (<font color='blue'><b>TV</b></font>):

In [8]:
class TV(object):
    def __init__(self, state=0):
        self.state = 0  # 0:Off; 1:On
        
    def on(self):
        if self.state == 0:
            self.state = 1
            print("Turn on TV")            
        else:
            print("TV is already on!")
            
    def off(self):
        if self.state == 0:
            print("TV is already off!")
        else:
            self.state = 0
            print("Turn off TV")
            

class CommandTurnOnTV(Command):
    def __init__(self, tv):
        self.tv = tv
        
    def do(self):
        self.tv.on()
        
        
class CommandTurnOffTV(Command):
    def __init__(self, tv):
        self.tv = tv
        
    def do(self):
        self.tv.off()

幾乎不用修改到任何 class 就可以滿足這個需求:

In [9]:
rctr = RemoteController()
tv = TV()

rctr.regr_command(
    'turn_on_tv',
    CommandTurnOnTV(tv)
)

rctr.regr_command(
    'turn_off_tv',
    CommandTurnOffTV(tv)
)

rctr.turn_on_tv
rctr.turn_off_tv

Turn on TV
Turn off TV


## <font color='darkblue'>Homework2</font>
接著我們來看 Homework2, 假設你經營一家餐廳 (<font color='blue'><b>Restaurant</b></font>), 並且雇用了一位服務生 (<font color='blue'><b>Waiter</b></font>) 與一位廚師 (<font color='blue'><b>Cooker</b></font>). 目前餐廳提供以下餐點:
* 牛排 (<font color='blue'><b>CommandOrderBeef</b></font>)
* 雞排 (<font color='blue'><b>CommandOrderChicken</b></font>)

客戶可以透過服務生 (<font color='blue'><b>Waiter</b></font>) 上的命令 <font color='blue'>giveOrder</font> 進行點餐. 點完餐後可以透過命令 <font color='blue'>sendOrder</font> 送出餐點給 廚師 (<font color='blue'><b>Cooker</b></font>) 進行備餐:

In [10]:
from abc import abstractmethod, ABC


class Cooker:
    def cookBeef(self):
        print("Beef is ready")
        return 'beef'
        
    def cookChicken(self):
        print("Chicken is ready")
        return 'chicken'

class Command(ABC):
    def do(self):
        raise NotImplemented()
        
        
class CommandOrderBeef(Command):
    def __init__(self, cooker):
        self.cooker = cooker
        
    def do(self):
        return self.cooker.cookBeef()
        

class CommandOrderChicken(Command):
    def __init__(self, cooker):
        self.cooker = cooker
        
    def do(self):
        return self.cooker.cookChicken()

        
class Waiter:
    def __init__(self):
        # Key as command name; value as command object
        self.cmd_dict = {}
        self.orders = []
        
    def giveOrder(self, order):
        self.orders.append(self.cmd_dict[order])
        
    def sendOrder(self):
        print("Send out order...")
        dish_list = []
        for o in self.orders:
            dish_list.append(o.do())
        
        self.orders = []
        return dish_list
        
        
class Restaurant:
    def __init__(self):
        self.waiter = Waiter()
        self.cooker = Cooker()
        self.waiter.cmd_dict['beef'] = CommandOrderBeef(self.cooker)
        self.waiter.cmd_dict['chicken'] = CommandOrderChicken(self.cooker)
        
    def start_business(self):
        self.waiter.giveOrder('beef')
        self.waiter.giveOrder('chicken')
        my_dish = self.waiter.sendOrder()
        return my_dish
        
    def start_business_with_new_dish(self):
        # TBD
        pass

接著來看執行結果:

In [11]:
r = Restaurant()
r.start_business()

Send out order...
Beef is ready
Chicken is ready


['beef', 'chicken']

在這個 Homework, 希望你為該餐廳提供新的菜單 豬排 (<font color='blue'><b>CommandOrderPork</b></font>) 並在營業 (<font color='brown'>修改上面函數</font> <font color='blue'>start_business_with_new_dish</font> <font color='brown'>並 新增/修改 需要的類別</font>) 後可以得到輸出 `['beef', 'pork', 'chicken']`:
```
r = Restaurant()
print(r.start_business_with_new_dish())  # ['beef', 'pork', 'chicken']
```

## <font color='darkblue'>Supplement</font>
* [Refactoring Guru - Strategy in Python](https://refactoring.guru/design-patterns/strategy/python/example)
> Strategy is a behavioral design pattern that turns a set of behaviors into objects and makes them interchangeable inside original context object.
* [Refactoring Guru- Command pattern](https://refactoring.guru/design-patterns/command)
* [IT邦幫忙: Day23 - 命令模式 | Command Pattern](https://ithelp.ithome.com.tw/articles/10246587)
* [NotFalse 技術客 - 命令模式 (Command Pattern)](https://notfalse.net/4/command-pattern)